In [ ]:
# 1. Preparing corpus - 콜금리 크롤링해서 .csv파일로 저장

import requests
from bs4 import BeautifulSoup
import re
f = open("/content/drive/MyDrive/callrate/call_rate.csv", "w")
f.write("date,callrate,\n")
for i in range(1,400): 
    url = 'https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL&page='+str(i)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    list_call = soup.select('div[class="section_exchange2"] tbody > tr')
    for call in list_call:
        date=str(call.select('td')[0].text.strip())
        call_index=str(call.select('td')[1].text.strip())
        f.write(str(date)+","+str(call_index)+'\n')
f.close()

In [ ]:
# 수집한 콜금리를 한달 전 콜금리와 비교해 up, down, same 라벨링한다.

import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
callrate = pd.read_csv('/content/drive/MyDrive/callrate/call_rate.csv')

callrate = callrate.sort_values(by=['date'])

# 비어있는 날짜(주말)와 금리를 추가하기
def all_date_range_df(date_range):

    df2 = pd.DataFrame({'date':date_range})
    
    return df2

date_range = (pd.date_range(start='20041201', end='20200903')).strftime("%Y.%m.%d").tolist()
df2 = all_date_range_df(date_range)

# 두 df를 병합하고 결측값 채우기
def df_outer_join_merge(df1, df2):
    
    df_OUTER_JOIN = pd.merge(df1, df2, left_on='date', right_on='date', how='outer', sort=True)
    
    # 공휴일에 없는 환율정보는 전날을 기준으로 결측값 채우기
    df_finally = df_OUTER_JOIN.fillna(method='ffill')
    
    return df_finally

fin_df = df_outer_join_merge(callrate, df2)
fin_df = fin_df.set_index('date')



import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
callrate = pd.read_csv('/content/drive/MyDrive/callrate/call_rate.csv')
callrate = callrate.sort_values(by=['date'])

# 한달 전 날짜 추가하기
month_date = []
for target_date in fin_df.index:
    month_before = (datetime.strptime(target_date,'%Y.%m.%d' ) - relativedelta(months = 1)).strftime('%Y.%m.%d')
    month_date.append(month_before)
fin_df['month_date'] = month_date

# 한달 전 날짜에 맞춰 콜 금리 추가하기
month_rate=[]
for target_date in fin_df['month_date']:
    try:
        rate = fin_df._get_value(target_date, 'callrate') 
        month_rate.append(rate)
    except: # keyError 제외
        month_rate.append(0.0)
fin_df['month_rate'] = month_rate

# 콜 금리의 차이를 구하고 0 초과 : 'up', 0 같음 : 'same', 0 미만 : 'down'으로 라벨링
value_diff = fin_df['callrate'] - fin_df['month_rate']
fin_df['label']='0'
for i, diff in enumerate(value_diff):
    if diff > 0:
        fin_df['label'][i] = 'up'
    elif diff == 0:
        fin_df['label'][i] = 'same'
    elif diff < 0:
        fin_df['label'][i] = 'down'   

# 2008년 ~ 2021년만 남기기
fin_df = fin_df.loc['2008.01.01':'2021.12.31', :]
fin_df.reset_index(inplace=True)
fin_df.to_csv('/content/drive/MyDrive/callrate/call_rate_final.csv',index=False)

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
callrate = pd.read_csv('/content/drive/MyDrive/callrate/call_rate.csv')
callrate = callrate.sort_values(by=['date'])

# 한달 전 날짜 추가하기
month_date = []
for target_date in fin_df.index:
    month_before = (datetime.strptime(target_date,'%Y.%m.%d' ) - relativedelta(months = 1)).strftime('%Y.%m.%d')
    month_date.append(month_before)
fin_df['month_date'] = month_date

# 한달 전 날짜에 맞춰 콜 금리 추가하기
month_rate=[]
for target_date in fin_df['month_date']:
    try:
        rate = fin_df._get_value(target_date, 'callrate') 
        month_rate.append(rate)
    except: # keyError 제외
        month_rate.append(0.0)
fin_df['month_rate'] = month_rate

# 콜 금리의 차이를 구하고 0 초과 : 'up', 0 같음 : 'same', 0 미만 : 'down'으로 라벨링
value_diff = fin_df['callrate'] - fin_df['month_rate']
fin_df['label']='0'
for i, diff in enumerate(value_diff):
    if diff > 0:
        fin_df['label'][i] = 'up'
    elif diff == 0:
        fin_df['label'][i] = 'same'
    elif diff < 0:
        fin_df['label'][i] = 'down'   

# 2008년 ~ 2021년만 남기기
fin_df = fin_df.loc['2008.01.01':'2021.12.31', :]
fin_df.reset_index(inplace=True)
fin_df.to_csv('/content/drive/MyDrive/callrate/call_rate_final.csv',index=False)